In [ ]:
import functools
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry
import skimage.draw
import skimage.filters

import tensorflow as tf

import pydicom

import pymedphys
import pymedphys._dicom.structure as dcm_struct

from names import names_map

In [ ]:
# Put all of the SASH DICOM data here, file structure doesn't matter:
data_path_root = pathlib.Path.home().joinpath('.data/dicom-ct-and-structures')

dcm_paths = list(data_path_root.rglob('**/*.dcm'))

In [ ]:
dcm_headers = []

for dcm_path in dcm_paths:
    dcm_headers.append(pydicom.read_file(
        dcm_path, force=True, specific_tags=['SOPInstanceUID', 'SOPClassUID']))

In [ ]:
ct_image_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "CT Image Storage"
}

In [ ]:
structure_set_paths = {
    header.SOPInstanceUID: path
    for header, path in zip(dcm_headers, dcm_paths)
    if header.SOPClassUID.name == "RT Structure Set Storage"
}

In [ ]:
# structure_uid = list(structure_set_paths.items())[0][0]
structure_uid = '1.2.840.10008.5.1.4.1.1.481.3.1574822743'

In [ ]:
structure_set_path = structure_set_paths[structure_uid]
structure_set_path

structure_set = pydicom.read_file(
    structure_set_path, 
    force=True, 
    specific_tags=['ROIContourSequence', 'StructureSetROISequence'])

In [ ]:
number_to_name_map = {
    roi_sequence_item.ROINumber: names_map[roi_sequence_item.ROIName]
    for roi_sequence_item in structure_set.StructureSetROISequence
    if names_map[roi_sequence_item.ROIName] is not None
}

number_to_name_map

In [ ]:
contours_by_ct_uid = {}

for roi_contour_sequence_item in structure_set.ROIContourSequence:
    try:
        structure_name = number_to_name_map[roi_contour_sequence_item.ReferencedROINumber]
    except KeyError:
        continue
        
    for contour_sequence_item in roi_contour_sequence_item.ContourSequence:
        ct_uid = contour_sequence_item.ContourImageSequence[0].ReferencedSOPInstanceUID
        
        try:
            _ = contours_by_ct_uid[ct_uid]
        except KeyError:
            contours_by_ct_uid[ct_uid] = dict()
                    
        try:
            contours_by_ct_uid[ct_uid][structure_name].append(contour_sequence_item.ContourData)
        except KeyError:
            contours_by_ct_uid[ct_uid][structure_name] = [contour_sequence_item.ContourData]

In [ ]:
# ct_uid = list(contours_by_ct_uid.keys())[50]
ct_uid = '1.2.840.113704.1.111.2804.1556591059.12956'

In [ ]:
ct_path = ct_image_paths[ct_uid]
dcm_ct = pydicom.read_file(ct_path, force=True)
dcm_ct.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

In [ ]:
def get_image_transformation_parameters(dcm_ct):
    # From Matthew Coopers work in ../old/data_generator.py
    
    position = dcm_ct.ImagePositionPatient
    spacing = [x for x in dcm_ct.PixelSpacing] + [dcm_ct.SliceThickness]
    orientation = dcm_ct.ImageOrientationPatient

    dx, dy, *_ = spacing
    Cx, Cy, *_ = position
    Ox, Oy = orientation[0], orientation[4]
    
    return dx, dy, Cx, Cy, Ox, Oy


In [ ]:
contours_by_ct_uid[ct_uid].keys()

In [ ]:
organ = 'rectum'

In [ ]:
original_contours = contours_by_ct_uid[ct_uid][organ]

In [ ]:
def reduce_expanded_mask(expanded_mask, img_size, expansion):
    return np.mean(np.mean(
        tf.reshape(expanded_mask, (img_size, expansion, img_size, expansion)),
        axis=1), axis=2)

In [ ]:
def calculate_aliased_mask(contours, dcm_ct, expansion=5):
    dx, dy, Cx, Cy, Ox, Oy = get_image_transformation_parameters(dcm_ct)
    
    ct_size = np.shape(dcm_ct.pixel_array)
    x_grid = np.arange(Cx, Cx + ct_size[0]*dx*Ox, dx*Ox)
    y_grid = np.arange(Cy, Cy + ct_size[1]*dy*Oy, dy*Oy)
    
    new_ct_size = np.array(ct_size) * expansion
    
    expanded_mask = np.zeros(new_ct_size)
    
    for xyz in contours:
        x = np.array(xyz[0::3])
        y = np.array(xyz[1::3])
        z = xyz[2::3]

        assert len(set(z)) == 1

        r = (((y - Cy) / dy * Oy)) * expansion + (expansion - 1) * 0.5
        c = (((x - Cx) / dx * Ox)) * expansion + (expansion - 1) * 0.5

        expanded_mask = np.logical_or(expanded_mask, skimage.draw.polygon2mask(new_ct_size, np.array(list(zip(r, c)))))
        
    mask = reduce_expanded_mask(expanded_mask, ct_size[0], expansion)
    mask = 2 * mask - 1
    
    return x_grid, y_grid, mask

In [ ]:
def get_contours_from_mask(x_grid, y_grid, mask):
    cs = plt.contour(x_grid, y_grid, mask, [0]);
    
    contours = [
        path.vertices for path in cs.collections[0].get_paths()
    ]
    
    plt.close()
    
    return contours

In [ ]:
x_grid, y_grid, mask_with_aliasing = calculate_aliased_mask(original_contours, dcm_ct)
_, _, mask_without_aliasing = calculate_aliased_mask(original_contours, dcm_ct, expansion=1)

In [ ]:
x_grid, y_grid, mask_with_shift = calculate_aliased_mask(np.array(original_contours) + 20, dcm_ct)

In [ ]:
# original_contours

In [ ]:
contours_with_aliasing = get_contours_from_mask(x_grid, y_grid, mask_with_aliasing)
contours_without_aliasing = get_contours_from_mask(x_grid, y_grid, mask_without_aliasing)

In [ ]:
plt.figure(figsize=(10,10))

for contour in contours_with_aliasing:
    plt.plot(contour[:,0], contour[:,1])
    plt.plot(contour[:,0], contour[:,1])

for xyz in original_contours:
    x = np.array(xyz[0::3])
    y = np.array(xyz[1::3])
    
    plt.plot(x, y)
    
plt.axis('equal')   

In [ ]:
plt.figure(figsize=(10,10))

for contour in contours_without_aliasing:
    plt.plot(contour[:,0], contour[:,1])
    plt.plot(contour[:,0], contour[:,1])

for xyz in original_contours:
    x = np.array(xyz[0::3])
    y = np.array(xyz[1::3])
    
    plt.plot(x, y)
    
plt.axis('equal')   

In [ ]:
def soft_surface_dice(reference, evaluation):
    edge_reference = skimage.filters.scharr(reference)
    edge_evaluation = skimage.filters.scharr(evaluation)
    
    score = (
        np.sum(np.abs(edge_evaluation - edge_reference)) /
        np.sum(edge_evaluation + edge_reference)
    )
    
    return score

In [ ]:
soft_surface_dice(mask_with_aliasing, mask_without_aliasing)

In [ ]:
soft_surface_dice(mask_with_aliasing, mask_with_aliasing)

In [ ]:
soft_surface_dice(mask_with_aliasing, mask_with_shift)

In [ ]:
mask_with_aliasing

In [ ]:
mask_without_aliasing

In [ ]:
edge_detection_with_aliasing = skimage.filters.scharr(mask_with_aliasing)
edge_detection_without_aliasing = skimage.filters.scharr(mask_without_aliasing)
edge_detection_with_shift = skimage.filters.scharr(mask_with_shift)

# mask_with_shift

In [ ]:
plt.contourf(x_grid, y_grid, edge_detection_with_aliasing)
plt.colorbar()
plt.axis('equal')

In [ ]:
plt.contourf(x_grid, y_grid, edge_detection_without_aliasing)
plt.colorbar()
plt.axis('equal')

In [ ]:
plt.contourf(x_grid, y_grid, edge_detection_with_shift)
plt.colorbar()
plt.axis('equal')

In [ ]:
(
    np.sum(np.abs(edge_detection_with_aliasing - edge_detection_without_aliasing)) /
    np.sum(edge_detection_with_aliasing + edge_detection_without_aliasing)
)

In [ ]:
(
    np.sum(np.abs(edge_detection_with_aliasing - edge_detection_with_shift)) /
    np.sum(edge_detection_with_aliasing + edge_detection_with_shift)
)